In [1]:
1+1

2

In [ ]:
%pip install torch torchvision 

In [ ]:
import torch 
x = torch.tensor([1.0, 2.0, 3.0]) # Tensor ist ein n-dimensionales Array
x

tensor([1., 2., 3.])

In [ ]:
random_tensor = torch.rand(2,3) # 2x3 Matrix mit Zufallswerten (gleichverteilt zwischen 0 und 1)
random_tensor

tensor([[0.5869, 0.8596, 0.9511],
        [0.1601, 0.9899, 0.4052]])

In [8]:
zeros_tensor = torch.zeros(2,3) # 2x3 Matrix mit Nullen
zeros_tensor

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [10]:
ones_tensor = torch.ones(2,3) # 2x3 Matrix mit Einsen
ones_tensor

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [11]:
x + ones_tensor 

tensor([[2., 3., 4.],
        [2., 3., 4.]])

In [14]:
# Matrixmultiplikation
a = torch.tensor([[1., 2,], [3,4]])
b = torch.tensor([[5., 6], [7, 8]])
torch.matmul(a,b) # oder a @ b
a @ b

tensor([[19., 22.],
        [43., 50.]])

In [ ]:
a*b 

tensor([[ 5., 12.],
        [21., 32.]])

In [ ]:
a.sum() # Summe aller Elemente in a

tensor(10.)

In [ ]:
a.mean() # Durchschnittswert aller Elemente in der Matrix a

tensor(2.5000)

In [60]:
if torch.cuda.is_available(): 
    device = torch.device("cuda") # GPU
else:
    device = torch.device("cpu") # CPU

device = torch.device("mps") # Mac mit Apple Silicon

In [18]:
# Gradienten berechnen 

x = torch.tensor(2.0, requires_grad=True) # Modus: Gradient wird mitgeschleppt 
y = x**2 + 3*x + 5 
# 2*x + 3 -> 2*2 + 3 = 7
y.backward() # Gradient berechnen
x.grad # Gradient ausgeben

tensor(7.)

In [37]:
import torch.nn as nn 
import torch.optim as optim 

# 10 Eingänge, 1 Ausgang -> Fully connected feed-forward neural network
model = nn.Linear(10,1) # im Tensorflow: Dense
# Perceptron
# in wiss. Publikationen: FC, FFN, FFNN, MLP (Multi Layer Perceptron)

loss = nn.MSELoss() # Mean Squared Error 
# loss, cost, criterion, crit, objective function, cost function 
optimizer = optim.SGD(model.parameters(), lr = 1e-2) # Stochastic Gradient Descent
# lr = Lernrate (eta, alpha); model.parameters() -> Gewichte+Bias 

input_data = torch.randn(10) # Zufallsinput
y = torch.tensor([1.0]) # Zielwert, target, out, output, regr, label, ground truth
output = model(input_data) # Vorwärtsdurchlauf, forward pass, inference, y_pred, y_hat 

loss_output = loss(output, y) # Unterschied zwischen Model und Realität 
                              # Anfangswert der Verlustfunktion

print(f"Kosten: {loss_output.item():.2f} (vor Update)") 

# Gradientenabstiegsmethoden/Optimierer
loss_output.backward() 
optimizer.step() # Update der Gewichte

output = model(input_data)
loss_output = loss(output, y)
print(f"Kosten: {loss_output.item():.2f} (nach Update)")

Kosten: 4.70 (vor Update)
Kosten: 2.61 (nach Update)


In [ ]:
# Hello World im Deep Learning 

# MNIST Datensatz: handgeschriebene Ziffern (0-9)
# Fashion-MNIST:  Kleidungsstücke (10 Produktkategorien) vom Zalando Online-Shop

from torchvision import datasets, transforms

transform = transforms.Compose([transforms.ToTensor(),])

train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True) 
# Größe der Mini-Batches: Zweierpotenzen (32, 64, 128, 256,...)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [47]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__() # Superklasse initialisieren
        self.layer1 = nn.Linear(28*28, 128) # Eingabeschicht -> versteckte Schicht (28x28 Pixel -> 128 Neuronen)
        self.layer2 = nn.Linear(128, 64)    # versteckte Schicht -> versteckte Schicht
        self.dropout = nn.Dropout(0.2)          # Dropout-Schicht 
        self.layer3 = nn.Linear(64, 10)    # versteckte Schicht -> Ausgabeschicht ( 10 Produktkategorien)
    
    def forward(self, x):
        x = x.view(-1, 28*28) # Flatten: 28x28 Pixel in Vektor umwandeln
        x = torch.relu(self.layer1(x)) # Aktivierungsfunktion ReLU
        x = torch.relu(self.layer2(x))
        x = self.dropout(x)  # Dropout anwenden
        x = self.layer3(x) # Keine Aktivierungsfunktion in der Ausgabeschicht (logits)
        return x
model = NN()

In [ ]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        return self.model(x)
    
model = NN()

In [61]:
model = NN().to(device) 

loss = nn.CrossEntropyLoss() # 10 Klassen 
optimizer = optim.SGD(model.parameters(), lr=1e-3)

num_epochs = 20 

for epoch in range(num_epochs): # Training-Loop, 
    # epoch = 1 mal hat Algorithmus Datensatz gesehen
    model.train() # Setzen des Modells in Trainingsmodus
    running_loss = 0.0 # Laufende Kosten
    for images, labels in train_loader: # Mini-Batches durchlaufen
        images, labels = images.to(device), labels.to(device) # 
        optimizer.zero_grad() # Gradient zurücksetzen
        # Vorwärtsdurchlauf
        outputs = model(images)
        curr_loss = loss(outputs, labels) # aktueller Wert der Kostenfunktion
        # Rückwärts
        curr_loss.backward() # Gradient berechnen
        optimizer.step()    # Gewichte updaten
        running_loss += curr_loss.item() # Kosten aufsummieren 
    
    print("Epoch [" + str(epoch+1) + "/" + str(num_epochs) + "], Loss:" + str(running_loss))


Epoch [1/20], Loss:2131.118208169937
Epoch [2/20], Loss:2051.8632428646088
Epoch [3/20], Loss:1890.6448457241058
Epoch [4/20], Loss:1632.6226350069046
Epoch [5/20], Loss:1384.3486088514328
Epoch [6/20], Loss:1213.8686493635178
Epoch [7/20], Loss:1094.782872736454
Epoch [8/20], Loss:1007.6372411251068
Epoch [9/20], Loss:944.1282235383987
Epoch [10/20], Loss:894.8432307243347
Epoch [11/20], Loss:855.1661875844002
Epoch [12/20], Loss:824.84856569767
Epoch [13/20], Loss:801.4100170731544
Epoch [14/20], Loss:780.0945774316788


KeyboardInterrupt: 

In [ ]:
model.eval() # Evaluierungsmodus
correct = 0
total = 0

with torch.no_grad(): # kein Gradient wird berechnet 
    for images, labels in test_loader:
        outputs = model(images)
        predicted = torch.max(outputs.data, 1)[-1] # argmax von outputs
        total += labels.size(0) # Gesamte Anzahl der Labels
        correct += (predicted==labels).sum().item() # Anzahl der richtig klassifizierten Labels

accuracy = correct/total
print("Accuracy: " + str(accuracy))

Accuracy: 0.74
